In [1]:
import pandas as pd
import random

### Individual Pathway: Glycolysis___Glucone.txt

**load the pathway**

In [10]:
glycolysis_glucone = pd.read_csv('pathwaycommons-individual/Glycolysis___Glucone.txt', sep='\t',header=None)
set(glycolysis_glucone[1].tolist())

{'catalysis-precedes',
 'consumption-controlled-by',
 'controls-production-of',
 'reacts-with',
 'used-to-produce'}

In [11]:
glycolysis_glucone.head()
# chebi is a chemical not a protein - different database

,0,1,2
0,ACSS2,catalysis-precedes,ADPGK
1,ACSS2,catalysis-precedes,DLAT
2,ACSS2,controls-production-of,chebi:15351
3,ACSS2,controls-production-of,chebi:16027
4,ADH1A,catalysis-precedes,ALDH3A1


In [12]:
glycolysis_glucone.drop(columns=[1],inplace=True)
print(len(glycolysis_glucone))
glycolysis_glucone.head()

454


,0,2
0,ACSS2,ADPGK
1,ACSS2,DLAT
2,ACSS2,chebi:15351
3,ACSS2,chebi:16027
4,ADH1A,ALDH3A1


**load the union ppi and ID map**

In [8]:
union_ppi = pd.read_csv('union_ppi.txt', sep='\t', header=None)
union_ppi.head()

,0,1,2,3
0,TACC1_HUMAN,RUXG_HUMAN,0.736771,U
1,TACC1_HUMAN,KAT2A_HUMAN,0.292198,U
2,TACC1_HUMAN,CKAP5_HUMAN,0.724783,U
3,TACC1_HUMAN,YETS4_HUMAN,0.542597,U
4,TACC1_HUMAN,LSM7_HUMAN,0.714823,U


In [16]:
id_map = pd.read_csv('mapped_ids_reviewed.csv', sep='\t')
id_map.head()

,gene_name,entry_name
0,PLA2G10,PA2GX_HUMAN
1,FDFT1,FDFT_HUMAN
2,UGCG,CEGT_HUMAN
3,CYP1A2,CP1A2_HUMAN
4,RARS2,SYRM_HUMAN


**map the inidividual pathway to entry_names using ID map**

In [17]:
mapping_dict = dict(zip(id_map["gene_name"], id_map["entry_name"]))

glycolysis_glucone[0] = glycolysis_glucone[0].replace(mapping_dict)
glycolysis_glucone[2] = glycolysis_glucone[2].replace(mapping_dict)

In [18]:
glycolysis_glucone.head()

,0,2
0,ACSA_HUMAN,ADPGK_HUMAN
1,ACSA_HUMAN,ODP2_HUMAN
2,ACSA_HUMAN,chebi:15351
3,ACSA_HUMAN,chebi:16027
4,ADH1A_HUMAN,AL3A1_HUMAN


**get 50% of the nodes as input nodes, then deterministically select the edges that contains these nodes from the PPI**

In [20]:
all_gg_nodes = list(set(glycolysis_glucone[0])) + list(set(glycolysis_glucone[2]))
len(all_gg_nodes)

141

In [28]:
sample_size = max(1, len(all_gg_nodes) // 4) # TODO: change this to multiplication 
# vary the percentage of nodes being sampled - GNN will generalize better then
gg_random_sample_1 = random.sample(all_nodes_gg, sample_size)
print(gg_random_sample_1)
mask = glycolysis_glucone[0].isin(gg_random_sample_1) & glycolysis_glucone[2].isin(gg_random_sample_1)
matching_rows = glycolysis_glucone[mask]
print(matching_rows)

['BPGM', 'chebi:17665', 'PFKP', 'chebi:44897', 'AKR1A1', 'ALDH3A2', 'DLAT', 'G6PC1', 'DLST', 'chebi:17720', 'chebi:30744', 'HK3', 'ADH1C', 'chebi:9532', 'PDHB', 'chebi:80404', 'ADH4', 'chebi:29042', 'ENO3', 'chebi:16108', 'HK1', 'chebi:17925', 'chebi:17719', 'chebi:15903', 'chebi:16027', 'chebi:17808', 'chebi:16001', 'chebi:28013', 'chebi:80219', 'PGM2', 'chebi:17835', 'ADH1A', 'chebi:32816', 'chebi:16084', 'GAPDH']
               0            2
120        G6PC1  chebi:16027
121        G6PC1  chebi:17665
122        G6PC1  chebi:17925
318  chebi:15903  chebi:16027
320  chebi:15903  chebi:17719
321  chebi:15903  chebi:17925
324  chebi:16001  chebi:17720
327  chebi:16027  chebi:15903
328  chebi:16027  chebi:17719
335  chebi:16084  chebi:28013
337  chebi:16108  chebi:28013
361  chebi:17665        G6PC1
372  chebi:17665  chebi:17719
373  chebi:17665  chebi:17925
374  chebi:17665  chebi:29042
377  chebi:17719  chebi:15903
378  chebi:17719  chebi:17665
380  chebi:17720  chebi:16001
388  chebi